![python logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)

In [11]:
# 这段代码定义了一个函数， 可以将相同的参数传给两个版本的函数后比较运行时间， 后面会用到， 默认重复100000次
from timeit import timeit

def func_compare(func_1, func_2, args = None, kwargs = None, n =1000000):
    if args is None:
        args =[]
    if kwargs is None:
        kwargs = {} 
        
    print("func_1 result:")
    print(func_1(*args, **kwargs))
    print()
    
    print("func_2 result:")
    print(func_2(*args, **kwargs)) 
    print()

    t1 = timeit(lambda : func_1(*args, **kwargs), number = n)
    t2 = timeit(lambda : func_2(*args, **kwargs), number = n)
    
    print('time: {t1:0.3f}s vs {t2:0.3f}s'.format(t1=t1, t2=t2))
    print('----------------------')

# 如何让代码更pythonic

充分利用python的特点和功能， 可以让我们的代码更加的简洁， 有更好的可读性， 很多情况下也会带来更好的性能。 符合这样要求的代码也被python社区称为pythonic的代码。

这个notebook主要举例一些使用python时遇到的场景。 针对这些场景， 给出两个版本的代码， 它们都实现了同样的功能， 但是第一个版本没有充分利用python特性，另一个版本则比较pythonic。

## 利用tuple/list unpack机制进行批量的赋值

### 1. 带有嵌套的tuple/list中的数值取出并赋值给多个变量

从一个tuple/list中将数据取出， 赋值给几个不同的变量是一个常见的需求。有时候这个tuple/list中还会嵌套更多的tuple/list。

下面的例子里， 我们有这样一个tuple

In [1]:
d = ('male',('Junjie', 'Cai'), 'cjj@jfpal.com', 'data engineer')

In [2]:
sex, (first_name, family_name), email, title = d

现在希望将里面的性别， 姓， 名， 邮箱几个数据取出并赋值给相应的变量后， 然后用这些变量组成一个字符串返回。 

In [223]:
def func_ver_1(d):
    sex = d[0]
    email = d[2]
    name = d[1]
    family_name = name[1]
    first_name = name[0]
    title = d[3]
    
    return "{family_name} {first_name} is a {sex} {title} with email:{email}".format(**locals())
    
def func_ver_2(d):
    sex, (first_name, family_name), email, title = d #注
    
    return "{family_name} {first_name} is a {sex} {title} with email:{email}".format(**locals())
    
d = ('male',('Junjie', 'Cai'), 'cjj@jfpal.com', 'data engineer')

func_compare(func_ver_1, func_ver_2, [d])

func_1 result:
Cai Junjie is a male data engineer with email:cjj@jfpal.com

func_2 result:
Cai Junjie is a male data engineer with email:cjj@jfpal.com

time: 2.962s vs 2.548s
----------------------


注:
    
只要让需要赋值的变量名， 组成和等号右边结构一致的tuple， 就能正确的批量赋值。 即使tuple/list内嵌套了别的tuple/list也没问题

### 2.在循环体中利用unpacking

现在有多个和上面的例子结构一样的tuple， 组成了一个list。

In [224]:
d_list = [
 ('male',('Junjie', 'Cai'), 'cjj@jfpal.com', 'data engineer'),
 ('male',('Hongfei', 'Bao'), 'bhf@jfpal.com', 'data engineer')    
]

现在需要循环体中将里面的性别， 姓， 名， 邮箱几个数据取出并赋值给相应的变量。

In [93]:
def func_ver_1(d_list):
    str_list = []
    
    for d in d_list:
        sex = d[0]
        email = d[2]
        name = d[1]
        family_name = name[1]
        first_name = name[0]
        title = d[3]
        
        str_list.append("{family_name} {first_name} is a {sex} {title} with email:{email}".format(**locals()))
        
    return str_list
    
    
def func_ver_2(d):
    str_list = []    
    for sex, (first_name, family_name), email, title in d_list:      
        str_list.append("{family_name} {first_name} is a {sex} {title} with email:{email}".format(**locals()))        
    
    return str_list

func_compare(func_ver_1, func_ver_2, [d_list])

func_1 result:
['Cai Junjie is a male data engineer with email:cjj@jfpal.com', 'Bao Hongfei is a male data engineer with email:bhf@jfpal.com']

func_2 result:
['Cai Junjie is a male data engineer with email:cjj@jfpal.com', 'Bao Hongfei is a male data engineer with email:bhf@jfpal.com']

time: 5.450s vs 4.767s
----------------------


### 3.只选取tuple中开头和结尾部分的变量
接第1个例子， 假设这里只需要取出第一个和最后一个变量

In [96]:
def func_ver_1(d):
    sex = d[0]
    email = d[2]
    
    return "Owner of {email} is {sex}".format(**locals())
    
def func_ver_2(d):
    sex, *_, email = d #注
    
    return "Owner of {email} is {sex}".format(**locals())    
    
d = ('male',('Junjie', 'Cai'), 'cjj@jfpal.com')

func_compare(func_ver_1, func_ver_2, [d])

func_1 result:
Owner of cjj@jfpal.com is male

func_2 result:
Owner of cjj@jfpal.com is male

time: 1.400s vs 1.658s
----------------------


注:

tuple/list unpacking是不能只选择头尾部分的数据的， 但是如果对tuple/list中间的部分不感兴趣， 可以用 **```*_```** 统一接收， 这样就会将中间段的数据放入变量名为**_**的list中， 后面代码不使用即可。

注意这里使用**_**作为变量名， 只是一种约定， 表示unpacking后，这个变量存的信息代码的作者并不关心。 不过实际上使用任何合法的变量名都是可以的。


### 4. 数值交换
有a, b两个变量， 

In [227]:
a = 1
b = 10

希望交换a, b两个变量的数值

In [228]:
def func_ver_1(a, b):
    temp = b
    b = a
    a = temp
    
    return (a, b)

def func_ver_2(a, b):
    a, b = b, a #注
    
    return (a, b)

func_compare(func_ver_1, func_ver_2, [a,b])

func_1 result:
(10, 1)

func_2 result:
(10, 1)

time: 0.396s vs 0.360s
----------------------


注释：

如果不是用在函数的参数列表， 不加()， python实际上也是创建了一个tuple哦

In [229]:
x = 1,2
print(type(x))

<class 'tuple'>


所以这里的数值交换实际上还是利用了tuple/list unpacking的机制。 这种场景很容易推广到多个变量数值进行交换的情况

In [230]:
a = 1
b = 2
c = 3

b, c, a = a, b, c

print(a, b, c)

3 1 2


## 字符串的连接

一个很常见的任务就是把list中的字符串用某个特定的字符进行拼接， 例如我们有这样一个list

In [12]:
s_list = ['To', 'be', 'or', 'not', 'to', 'be']

希望把他们用空格去连接几个单词组成一个句子

In [13]:
def func_1(s_list, sep):
    string = ''
    for i, s in enumerate(s_list):
        if i!=0:
            string += sep+s
        else:
            string += s
    return string

def func_2(s_list,sep):
    return sep.join(s_list)


sep = ' '
func_compare(func_1, func_2, [s_list, sep])

func_1 result:
To be or not to be

func_2 result:
To be or not to be

time: 1.512s vs 0.481s
----------------------


第二个版本中， 用python内置的函数比自己去实现， 简洁度， 效率都高了不是一点点。

## 二. 快速创建拥有相同值的list

某时候我们需要快速的创建一个list， 里面的值是相同的。 

例如， 我们创建一个长度为4的list, 其中每个位置的内容都是[0,0,0,0] 

In [234]:
def func_1(v, n):
    l = []
    for i in range(n):
        l.append(v)
    return l

def func_2(v, n):
    return [v]*n

func_compare(func_1, func_2, [[0,0,0,0],4])

func_1 result:
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

func_2 result:
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

time: 1.206s vs 0.554s
----------------------


python提供的**```*```**运算符可以将list复制扩增。不过这里要注意的是, 这种方法创建的list中， 每个位置指向的都是同一个数据。 用下面的代码可以验证这点

In [20]:
x = [0]*4
x[0] = 5
x

[5, 0, 0, 0]

In [16]:
a = [[0,0,0,0]]*4
a

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [18]:
print(a[0] is a[1]) #注1

a[0][0] = 10
print(a) #注2

True
[[10, 0, 0, 0], [10, 0, 0, 0], [10, 0, 0, 0], [10, 0, 0, 0]]


注释
1. **is**运算符的结果为True， 说明a[0], a[1]其实指向的数据是内存中同一位置的数据
2. 由于a[0], a[1], a[2], a[3]指向的是内存中同一位置的数据， 因此修改了a[0]后, a[1], a[2], a[3]也跟着变了

如果真的只想修改a[0], 可以用以下的代码来实现

In [19]:
a = [[0,0,0,0]]*4

b = a[0].copy()
b[0] = 10 #注1

a[0] = b #注2
print(a)

[[10, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]


注释

1. 由于b是从a[0] copy出来的，因此它的内容和a[0]一样， 但是在内存中和a[0]是不一样位置的数据, 修改b， 不会影响a[0]指向的数据。
2. 让a[0]重新指向b

## 判断for循环是否全部通过

有时候我们要判断一个for循环是否中途被break掉还是“寿终正寝”。 

例如下面判断list_a中每一个元素是否都属于list_b(这里暂时封印一下set...)

In [235]:
list_a = ['java','spark','python']
list_b = ['java','html5','excel','spark','python']

In [236]:
def func_1(list_a, list_b):
    is_all_in = True
    
    for x in list_a:
        if not x in list_b:
            is_all_in = False
            break
            
    if is_all_in:
        return 'all item in list_a is in list_b'
    else:
        return 'not all item in list_a is in list_b'        
        
def func_2(available, demand):
    for x in list_a:
        if not x in list_b:
            return 'not all item in list_a is in list_b'        
    else: #注1
        return 'all item in list_a is in list_b'
        
func_compare(func_1, func_2, [list_a, list_b])

func_1 result:
all item in list_a is in list_b

func_2 result:
all item in list_a is in list_b

time: 0.746s vs 0.681s
----------------------


注释:
1. for ...else...中的else刚接触会觉得有点别扭， 总之记住如果for循环部分执行了break， 或者出现Exception， else分支就不会被执行即可。 利用这个语句可以省掉一个flag去标记for循环是否中途被break掉。

当然， 这里只是为了举例需要， 真的需要判断list_a的所有元素是否在list_b中， 用set更简单。

In [146]:
list_a = ['java','spark','python']
list_b = ['java','html5','excel','spark','python']

print(set(list_a)<=set(list_b)) #注

True


```<=```用在set上， 可以用于判断一个集合是否是另一个集合的子集

## 10>c>b>a>0

如何判断10>c>b>a>0?

In [23]:
def func_ver_1(a,b,c):
    return (10>c) and (c>b) and (b>a) and (a>0)
        

def func_ver_2(a, b, c):
    return 10>c>b>a>0 #注1

a = 1
b = 3
c = 5

func_compare(func_ver_1, func_ver_2, [a, b, c])

func_1 result:
True

func_2 result:
True

time: 0.408s vs 0.409s
----------------------


注释:

1. 几乎每个学C语言的同学都会被老师告诫不能这么写， 不过这种写法在python确是完全可以的哦。

## 判断空容器和空值

### 1.判断容器为空

下面的例子里， 我们检查一个list是否为空， 如果是， 返回True， 否则返回False

In [24]:
def func_ver_1(l):
    if len(l)>0:
        return True
    else:
        return False
        
def func_ver_2(l):
    if l: #注1
        return True
    else:
        return False      
        
l = []

func_compare(func_ver_1, func_ver_2, [[]])

func_1 result:
False

func_2 result:
False

time: 0.379s vs 0.315s
----------------------


注:
1. 如果把list直接用在if语句中， 那么如果它们是空的， 对于if相当于False， 否则为True。 因此如果目的是判断它们是否为空， 就不用额外的使用len求长度了。 这点同样适用于tuple, set, dict, string, 见下面代码

In [25]:
if ():
    pass
else:
    print('tuple is empty')
    
if set():
    pass
else:    
    print('set is empty')
    
if {}:
    pass
else:    
    print('dict is empty')
    
if "":
    pass
else:    
    print('string is empty')    

tuple is empty
set is empty
dict is empty
string is empty


不过很遗憾， 对于数据分析常用的pandas.DataFrame和pandas.Series， 并不支持这个特征， 直接放在if里会报错。

In [241]:
from pandas import DataFrame
if DataFrame():
    pass
else:
    print('DataFrame is empty')   

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### 2. 判断数据是否是None

这里假设数据是None返回False, 否则返回Yes

In [242]:
def func_ver_1(l):
    if x is None:
        return True
    else:
        return False
        
def func_ver_2(l):
    if x: #注1
        return False
    else:
        return True
        
x = None

func_compare(func_ver_1, func_ver_2, [[]])

func_1 result:
True

func_2 result:
True

time: 0.406s vs 0.334s
----------------------


注释：

1. 如果将None直接用在if中， 相当于False， 因此可以利用这点很方便的判断一个数据是否为None

## 从一个list/ dict筛选，变换成另一个list/dict

### 1. list快速变换
假设下面把list中的偶数取出， 求平方后存在另一个list

In [243]:
l = [1,2,3,4,5,6,7,8,9,10]

In [28]:
def func_ver_1(l):
    new_list = []
    for v in l:
        if v%2==0:
            new_list.append(v**2)
    return new_list

def func_ver_2(l):
    return [v**2 for v in l if v%2==0] #注1



func_compare(func_ver_1, func_ver_2, [l])

func_1 result:
[]

func_2 result:
[]

time: 0.374s vs 0.485s
----------------------


注
1. 这里利用了python提供的list comprehension功能

### 2. dict筛选变换

下面的场景是有这样一个dict

In [30]:
d = {1:'apple', 2:'peach', 3:'melon', 4:'banana'}

把key为偶数的item取出后， 将value变成大写， 然后将这个key-value pair存入新的dict

In [248]:
def func_ver_1(d):
    new_dict = {}
    for k in d:
        if k%2==0:
            new_dict[k] = d[k].upper()
    return new_dict            
    
def func_ver_2(d):
    return {k:v.upper() for k,v in d.items() if k%2==0} #注1

d = {1:'apple', 2:'peach', 3:'melon', 4:'banana'}
func_compare(func_ver_1, func_ver_2, [d])

func_1 result:
{2: 'PEACH', 4: 'BANANA'}

func_2 result:
{2: 'PEACH', 4: 'BANANA'}

time: 1.388s vs 1.543s
----------------------


注释:

1. 这里需要同时遍历dict的key， value, 因此需要对d调用items()函数。 生成一个能生成包含key, value构成的tuple的dict_items类型

In [252]:
iterator = d.items()
print(iterator)

dict_items([(1, 'apple'), (2, 'peach'), (3, 'melon'), (4, 'banana')])


## 用两个list创建一个dict

一个list提供key, 另一个list提供value

In [36]:
def func_1(l_key, v_key):
    d = {}
    for i, v in enumerate(l_key):
        d[v] = v_key[i]
    return d

def func_2(l_key, v_key):
    return dict(zip(l_key, v_key)) #注1



func_compare(func_1, func_2, [l_key, l_value])

func_1 result:
{'b': 2, 'c': 3, 'a': 1}

func_2 result:
{'b': 2, 'c': 3, 'a': 1}

time: 0.923s vs 1.191s
----------------------


注释

1. 这里利用zip函数将两个list合并成一个generator, generator会按顺序输出一系列tuple, tuple中的数据分别来自两个list. 见下面代码。

In [205]:
list_a = ['a','b','c']
list_b = [1,2,3]

z = zip(list_a, list_b)
print(next(z))
print(next(z))
print(next(z))

('a', 1)
('b', 2)
('c', 3)


之后dict()函数从这一系列tuple中的取一项作为key, 第二项作为value

##  确保打开的资源被正确关闭

下面的场景里， 我们要确保打开了一个文件后， 要确保它们被关闭

In [209]:
def func_ver_1(file_name):
    try:
        f = open(file_name, 'w')        
        #对f做所需的操作， 这里略过
    except Exception as e:
        pass
    finally:
        f.close()

def func_ver_2(file_name):
    with open(file_name, 'w') as f: #注1
        #对f做所需的操作， 这里略过        
        pass
    
file_name = 'test_temp_file.txt'
func_compare(func_ver_1, func_ver_2, [file_name])


func_1 result:
None

func_2 result:
None

time: 25.202s vs 25.910s
----------------------


注释

1. 在with语句去打开文件， 将对这个文件所需的处理语句放在with中， 这样可以确、无论这些语句是否raise Exception， context manager都能保证文件都能被关闭。 可以省掉一大块try, except, finnaly组合， 让代码简洁不少。

    其他一些连接数据资源， 比如连接数据库的函数， 一般也会支持这种用法。 



## 判断数据的类型

下面我们判断a是否是tuple或list, 是的话返回True, 否则False

In [37]:
a = (1, 2, 3)

In [ ]:
判断函数可以用callable

In [49]:
def a():
    pass

callable(a)

True

In [212]:
def func_ver_1(a):
    if (str(type(a)) == "<class 'tuple'>") or (str(type(a)) == "<class 'list'>"):
        return True
    else:
        return False
    
def func_ver_2(a):
    if isinstance(a, (tuple, list)): #注1
        return True
    else:
        return False
    

func_compare(func_ver_1, func_ver_2, [a])

func_1 result:
True

func_2 result:
True

time: 1.303s vs 0.624s
----------------------


注释

1. 由于很多python教材都会在早期的时候引入对type函数的介绍， 因此初学者可能会很长时间内使用type去判断数据的类型。 不过isinstance是更好的选择。

# 应对不合法的输入参数


有时候我们需要考虑到输入的参数可能不合法的问题。 不过与其每次都检查一下输入参数的类型， 不妨将这个问题交给try, except机制去处理。

下面的例子里， 我们实现一个对两个数值求sum的功能， 如果输入参数并不能参与sum计算， 那么返回None

In [50]:

def func_ver_1(a, b):
    if isinstance(a, (float,int)) and isinstance(b, (float, int)):
        return sum([a, b])
    else:
        return None

        
def func_ver_2(a, b):
    try:
        return sum([a, b])
    except TypeError:
        return None
    
func_compare(func1, func2, [5,10])

NameError: name 'func1' is not defined

第二个版本的代码中， 我们不去主动检验输入参数的合法性， 而是由try, except去处理可能发生的问题， 简化了代码。 

而且由于类型判断本身也是一个耗时的步骤， 第二个版本的函书省略了类型判断的步骤， 效率也更高。

# 跨行的字符串连接

In [51]:
my_very_big_string = (
    "For a long time I used to go to bed early. Sometimes, "
    "when I had put out my candle, my eyes would close so quickly "
    "that I had not even time to say “I’m going to sleep.”"
)

my_very_big_string

'For a long time I used to go to bed early. Sometimes, when I had put out my candle, my eyes would close so quickly that I had not even time to say “I’m going to sleep.”'